In [1]:
# this file shows what the symbolic music data looks like 
# as well as the tokenization process we use for finetuning on CoLA (single sentence)
# and MRPC (double sentence) datasets

!git clone https://github.com/wazenmai/MIDI-BERT.git
%cd MIDI-BERT
!pip install -r requirements.txt
%cd MidiBERT
%cd CP

Cloning into 'MIDI-BERT'...
remote: Enumerating objects: 774, done.
remote: Counting objects: 100% (167/167), done.
remote: Compressing objects: 100% (135/135), done.
remote: Total 774 (delta 78), reused 81 (delta 32), pack-reused 607
Receiving objects: 100% (774/774), 75.34 MiB | 10.08 MiB/s, done.
Resolving deltas: 100% (392/392), done.
Checking out files: 100% (153/153), done.
/content/MIDI-BERT
     |████████████████████████████████| 11.2 MB 4.7 MB/s 
     |████████████████████████████████| 51 kB 7.1 MB/s 
     |████████████████████████████████| 22.3 MB 1.3 MB/s 
     |████████████████████████████████| 1.9 MB 47.7 MB/s 
     |████████████████████████████████| 2.5 MB 41.6 MB/s 
     |████████████████████████████████| 831.4 MB 2.5 kB/s 
     |████████████████████████████████| 895 kB 54.5 MB/s 
     |████████████████████████████████| 3.3 MB 34.8 MB/s 
     |████████████████████████████████| 895 kB 47.1 MB/s 
     |████████████████████████████████| 5.6 MB 44.3 MB/s 
  Created wheel for

/content/MIDI-BERT/MidiBERT
/content/MIDI-BERT/MidiBERT/CP


In [2]:
import numpy as np
import random
import pickle
import os
root = '../../data/CP'
dataset = 'pop909' # dataset containing Pop music; used for melody classification
data = np.load(os.path.join(root, f'{dataset}_train.npy'), allow_pickle=True)
print ("This is what the symbolic music data looks like")
print (data)
# token types: [Bar, Position, Pitch, Duration]
# Number of possible combinations for each token type: [4,18,88,66]
# the vectors at the end of the training set are all [ 2 16 86 64], which corresponds to the pad token

This is what the symbolic music data looks like
[[[ 0  0 55  3]
  [ 1  0 24 13]
  [ 1  2 54  3]
  ...
  [ 1  3 43  5]
  [ 1  5 57  1]
  [ 1  5 31  3]]

 [[ 1  6 38  3]
  [ 1  6 36  1]
  [ 1  7 43  3]
  ...
  [ 1  9 33  3]
  [ 1 10 38  1]
  [ 1 11 38  5]]

 [[ 1 11 42  5]
  [ 1 12 43  1]
  [ 1 14 50  9]
  ...
  [ 1  0 50 11]
  [ 1  0 23 13]
  [ 1  0 35  3]]

 ...

 [[ 0  0 18 11]
  [ 1  0 30  3]
  [ 1  0 37  5]
  ...
  [ 1 11 41  1]
  [ 1 11 44  1]
  [ 1 12 37  3]]

 [[ 1 13 54  3]
  [ 1 13 42  1]
  [ 1 13 46  1]
  ...
  [ 1  0 39  7]
  [ 1  0 42  7]
  [ 1  0 46  1]]

 [[ 1  0 47  7]
  [ 1  1 27  1]
  [ 1  2 56  1]
  ...
  [ 2 16 86 64]
  [ 2 16 86 64]
  [ 2 16 86 64]]]


In [3]:
# show the tokenization process for CoLA
!pip install datasets==1.11.0

from datasets import load_dataset

max_seq_length = 512
actual_task =  "cola"
dataset = load_dataset("glue", actual_task)

import pandas as pd

df = pd.DataFrame(dataset["train"][:])

print('Number of training sentences: {:,}\n'.format(df.shape[0]))

# Display 10 random rows from the data
df.sample(10)

     |████████████████████████████████| 264 kB 5.1 MB/s 
     |████████████████████████████████| 212 kB 53.1 MB/s 
     |████████████████████████████████| 134 kB 43.4 MB/s 


Downloading:   0%|          | 0.00/7.78k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.47k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/377k [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.
Number of training sentences: 8,551



,sentence,label,idx
1639,Bill left when it is certain that no one else ...,1,1639
2424,This movie just watches itself.,0,2424
6993,I am hoping to get an invitation and optimisti...,1,6993
5657,Cat ate the spider.,0,5657
3277,In price soared oil.,0,3277
2357,I sought the woods for game.,0,2357
8513,The pig grunts,1,8513
5797,I loved the policeman the baker intensely with...,0,5797
3446,All native speakers have a grammatical compete...,1,3446
668,I expect John to win and Harry to lose.,1,668


In [4]:
import torch
from transformers import BertTokenizer

sentences = df.sentence.values
labels = df.label.values

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

input_ids = []


for sent in sentences:

    encoded_dict = tokenizer.encode(
                        sent,                      # Sentence to encode.
                        max_length = max_seq_length ,           # Pad & truncate all sentences.
                        add_special_tokens = False,       # Pad is the only special token
                        truncation=True,
                        pad_to_max_length = True,
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
  
    input_ids.append(encoded_dict)
    

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
labels = torch.tensor(labels)

print('Original: ', sentences[0])
print('Tokenized: ', tokenizer.convert_ids_to_tokens(input_ids[0]))
print('Token IDs:', input_ids[0]) 

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2132: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Original:  Our friends won't buy this analysis, let alone the next one we propose.
Tokenized:  ['Our', 'friends', 'won', "'", 't', 'buy', 'this', 'analysis', ',', 'let', 'alone', 'the', 'next', 'one', 'we', 'propose', '.', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[P

In [5]:
# now convert the token IDs to 4 dimensional vectors

x = input_ids.numpy()
y_train = labels.numpy()
X_train = np.empty((x.shape[0], max_seq_length, 4))

# token types: [Bar, Position, Pitch, Duration]
# Number of possible combinations for each token type: [4,18,88,66]

#PAD:       [ 2 16 86 64]
#MASK:      [ 3 17 87 65]

#The total number of possible combinations (excluding pad and mask) is thus 2*16*86*64 = 176,128
#To constrain this closer to the vocabulary size of English BERT (roughly 30,000),
#we don’t map to zero values in the Bar column since they correspond to the beginning of a musical bar
#We also don’t map to values greater than 32 in the Duration column, since duration is measured in 32nd notes,
#and there are very few data points in the pretraining sets that have a duration greater than 32

# use modular arithmetic to map each ID to a unique 4D vector

X_train[:,:,0] = 1                                          # map to 1 in the Bar column
X_train[:,:,1] = x%16                                       # map to 0-16 in the Position column
X_train[:,:,2] = ((x/16).astype(int))%86                    # map to 0-86 in the Pitch column
X_train[:,:,3] = ((x/(16*86)).astype(int))%32               # map to 0-32 in the Duration column

# the pad token is initially mapped to [1 0 0 0]
# but we want it mapped to [ 2 16 86 64]
for i in range (0,x.shape[0]):
  for j in range (0,max_seq_length):
    X_train[i,j,:] = np.array([2, 16, 86, 64]) if(np.sum(X_train[i,j,:]) == 1) else X_train[i,j,:]
                              
X_train = X_train.astype(int)      
print('X_train shape: {}, y_train shape: {}'.format(X_train.shape,y_train.shape))     
print (" original ids")
print (x)       
print (" 4d vectors")  
print (X_train)

X_train shape: (8551, 512, 4), y_train shape: (8551,)
 original ids
[[ 3458  2053  1281 ...     0     0     0]
 [ 1448  1167 23563 ...     0     0     0]
 [ 1448  1167 23563 ...     0     0     0]
 ...
 [ 1135  1110  3123 ...     0     0     0]
 [  146  1125  1103 ...     0     0     0]
 [ 1327  1155  1225 ...     0     0     0]]
 4d vectors
[[[ 1  2 44  2]
  [ 1  5 42  1]
  [ 1  1 80  0]
  ...
  [ 2 16 86 64]
  [ 2 16 86 64]
  [ 2 16 86 64]]

 [[ 1  8  4  1]
  [ 1 15 72  0]
  [ 1 11 10 17]
  ...
  [ 2 16 86 64]
  [ 2 16 86 64]
  [ 2 16 86 64]]

 [[ 1  8  4  1]
  [ 1 15 72  0]
  [ 1 11 10 17]
  ...
  [ 2 16 86 64]
  [ 2 16 86 64]
  [ 2 16 86 64]]

 ...

 [[ 1 15 70  0]
  [ 1  6 69  0]
  [ 1  3 23  2]
  ...
  [ 2 16 86 64]
  [ 2 16 86 64]
  [ 2 16 86 64]]

 [[ 1  2  9  0]
  [ 1  5 70  0]
  [ 1 15 68  0]
  ...
  [ 2 16 86 64]
  [ 2 16 86 64]
  [ 2 16 86 64]]

 [[ 1 15 82  0]
  [ 1  3 72  0]
  [ 1  9 76  0]
  ...
  [ 2 16 86 64]
  [ 2 16 86 64]
  [ 2 16 86 64]]]


In [6]:
# now demonstrate the tokenization process for MRPC

max_seq_length = 512
actual_task =  "mrpc"
dataset = load_dataset("glue", actual_task)

import pandas as pd

df = pd.DataFrame(dataset["train"][:])

print('Number of training sentences: {:,}\n'.format(df.shape[0]))

df.sample(10)

Downloading: 0.00B [00:00, ?B/s]

Downloading: 0.00B [00:00, ?B/s]

Downloading: 0.00B [00:00, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.
Number of training sentences: 3,668



,sentence1,sentence2,label,idx
942,"Germany , another opponent of the war currentl...","Germany , another opponent of the war currentl...",1,1051
1328,"American has laid off 6,500 of its Missouri em...","Since October 2001 , American has laid off 6,1...",1,1481
2687,"Watson , of Whitakers , N.C. , was found guilt...","Dwight Watson , 50 , was convicted of making a...",1,2985
3130,Nordstrom also is expanding its play area in t...,Nordstrom also is expanding its play area in t...,1,3479
569,Preliminary reports were that the men were not...,The men had Pakistani passports and reportedly...,0,638
3653,The layoffs at Eddie Bauer headquarters were d...,The eliminations at Eddie Bauer come at all di...,1,4061
2045,"Leung , who faces a maximum of 50 years in pri...","Smith , who faces a maximum 40 years in prison...",0,2271
2538,"By Wednesday , the Red Planet will come as nea...","On Wednesday , at 7.51pm , Mars will be the cl...",1,2825
3032,Russian cosmonaut Malenchenko managed a space ...,Russian cosmonaut Malenchenko achieved a first...,1,3366
2842,Bush said that he ordered the Treasury Departm...,Bush said in a statement that he ordered the U...,1,3154


In [7]:
sentences1 = df.sentence1.values
sentences2 = df.sentence2.values
sentences = df[["sentence1", "sentence2"]].values.astype("str")
labels = df.label.values

from transformers import BertTokenizer

import torch

input_ids = []
attention_masks = []

for sent in sentences:
    encoded_dict = tokenizer.encode(
                        sent[0], sent[1],                                # Sentences to encode.
                        max_length = max_seq_length,           # Pad & truncate all sentences.
                        add_special_tokens = False,       # Pad is the only special token
                        truncation=True,
                        pad_to_max_length = True,
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    input_ids.append(encoded_dict)
    # we don't use segment ID encodings

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
labels = torch.tensor(labels)

print(' Original 1: ', sentences1[0])
print(' Original 2: ', sentences2[0])
print('Tokenized: ', tokenizer.convert_ids_to_tokens(input_ids[0]))
print('Token IDs:', input_ids[0]) 

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2132: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


 Original 1:  Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .
 Original 2:  Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .
Tokenized:  ['Am', '##ro', '##zi', 'accused', 'his', 'brother', ',', 'whom', 'he', 'called', '"', 'the', 'witness', '"', ',', 'of', 'deliberately', 'di', '##sto', '##rting', 'his', 'evidence', '.', 'Re', '##fer', '##ring', 'to', 'him', 'as', 'only', '"', 'the', 'witness', '"', ',', 'Am', '##ro', '##zi', 'accused', 'his', 'brother', 'of', 'deliberately', 'di', '##sto', '##rting', 'his', 'evidence', '.', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]',

In [8]:
x = input_ids.numpy()
y_train = labels.numpy()
X_train = np.empty((x.shape[0], max_seq_length, 4))

# token types: [Bar, Position, Pitch, Duration]
# Number of possible combinations for each token type: [4,18,88,66]

#PAD:       [ 2 16 86 64]
#MASK:      [ 3 17 87 65]

#The total number of possible combinations (excluding pad and mask) is thus 2*16*86*64 = 176,128
#To constrain this closer to the vocabulary size of English BERT (roughly 30,000),
#we don’t map to zero values in the Bar column since they correspond to the beginning of a musical bar
#We also don’t map to values greater than 32 in the Duration column, since duration is measured in 32nd notes,
#and there are very few data points in the pretraining sets that have a duration greater than 32

# use modular arithmetic to map each ID to a unique 4D vector

X_train[:,:,0] = 1                                          # map to 1 in the Bar column
X_train[:,:,1] = x%16                                       # map to 0-16 in the Position column
X_train[:,:,2] = ((x/16).astype(int))%86                    # map to 0-86 in the Pitch column
X_train[:,:,3] = ((x/(16*86)).astype(int))%32               # map to 0-32 in the Duration column

# the pad token is initially mapped to [1 0 0 0]
# but we want it mapped to [ 2 16 86 64]
for i in range (0,x.shape[0]):
  for j in range (0,max_seq_length):
    X_train[i,j,:] = np.array([2, 16, 86, 64]) if(np.sum(X_train[i,j,:]) == 1) else X_train[i,j,:]
                              
X_train = X_train.astype(int)      
print('X_train shape: {}, y_train shape: {}'.format(X_train.shape,y_train.shape))     
print (" original ids")
print (x)       
print (" 4d vectors")  
print (X_train)

X_train shape: (3668, 512, 4), y_train shape: (3668,)
 original ids
[[ 7277  2180  5303 ...     0     0     0]
 [10684  2599  9717 ...     0     0     0]
 [ 1220  1125  1502 ...     0     0     0]
 ...
 [  107  1284  1138 ...     0     0     0]
 [ 1109  1136  5783 ...     0     0     0]
 [ 1109  1476   118 ...     0     0     0]]
 4d vectors
[[[ 1 13 24  5]
  [ 1  4 50  1]
  [ 1  7 73  3]
  ...
  [ 2 16 86 64]
  [ 2 16 86 64]
  [ 2 16 86 64]]

 [[ 1 12 65  7]
  [ 1  7 76  1]
  [ 1  5  5  7]
  ...
  [ 2 16 86 64]
  [ 2 16 86 64]
  [ 2 16 86 64]]

 [[ 1  4 76  0]
  [ 1  5 70  0]
  [ 1 14  7  1]
  ...
  [ 2 16 86 64]
  [ 2 16 86 64]
  [ 2 16 86 64]]

 ...

 [[ 1 11  6  0]
  [ 1  4 80  0]
  [ 1  2 71  0]
  ...
  [ 2 16 86 64]
  [ 2 16 86 64]
  [ 2 16 86 64]]

 [[ 1  5 69  0]
  [ 1  0 71  0]
  [ 1  7 17  4]
  ...
  [ 2 16 86 64]
  [ 2 16 86 64]
  [ 2 16 86 64]]

 [[ 1  5 69  0]
  [ 1  4  6  1]
  [ 1  6  7  0]
  ...
  [ 2 16 86 64]
  [ 2 16 86 64]
  [ 2 16 86 64]]]
